<a href="https://colab.research.google.com/github/Ayushpatel2026/cnn-malaria/blob/main/MalariaDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

## **Data Preparation**

In [6]:

# load the data set and its info
dataset, dataset_info = tfds.load('malaria', with_info=True,
                                  as_supervised=True,
                                  shuffle_files=True,
                                  split=['train'] )

In [ ]:
# the dataset is of 27558 cell images
# it is given as a dictionary with the keys 'image' and 'label'
# each image is 103 by 103 pixels by 3 (for rgb)
# it is not split already, we need to do that below
# # let us look at the data
for data in dataset[0].take(1):
  print(data)

In [30]:
def split_dataset(dataset, train_ratio, val_ratio, test_ratio):
  DATASET_SIZE = len(dataset)

  # take from 0 to train_ratio*dataset_size
  train_dataset = dataset.take(int(train_ratio*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(train_ratio*DATASET_SIZE))

  # after removing train, take val_ratio*dataset_size
  val_dataset = val_test_dataset.take(int(val_ratio*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(val_ratio*DATASET_SIZE))

  return train_dataset, val_dataset, test_dataset

In [31]:


TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset, val_dataset, test_dataset = split_dataset(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)
